In [1]:
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
import serial
import time

In [2]:
# Among Us
print(chr(sum(range(ord(min(str(not())))))))

ඞ


In [3]:
model_path = "./converted_savedmodel/model.savedmodel"
label_path = "./converted_savedmodel/labels.txt"

In [4]:
model = tf.saved_model.load(model_path)
with open(label_path, 'r') as f:
  labels = f.read().splitlines()

In [5]:
print("Model Inputs:", model.signatures['serving_default'].structured_input_signature)
print("Model Outputs:", model.signatures['serving_default'].structured_outputs)

Model Inputs: ((), {'sequential_13_input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='sequential_13_input')})
Model Outputs: {'sequential_15': TensorSpec(shape=(None, 4), dtype=tf.float32, name='sequential_15')}


In [6]:
def predict_picture(frame):
  img = Image.fromarray(frame).resize((224, 224))
  img_array = np.array(img) / 255.0
  img_array = img_array.astype(np.float32)
  img_array = np.expand_dims(img_array, axis=0)
  try:
    infer = model.signatures['serving_default']
    predictions = infer(sequential_13_input=tf.constant(img_array))
    predicted_scores = predictions['sequential_15'].numpy()
    return np.argmax(predicted_scores)
  except Exception as e:
    print("Exception: ", e)
    return None

In [14]:
def send_serial(serial_port, num):
  byte = f"{num}".encode()
  serial_port.write(byte)
  print(f"Serial sent {byte}")

In [8]:
def capture_camera(camera_id, serial_port):
  window_name = "Camera Feed"
  
  cap = cv2.VideoCapture(camera_id)

  print("Press the space bar to capture an image. Press 'q' to exit.")

  while True:
    ret, frame = cap.read()
    if not ret:
      print("Failed to capture frame.")
      break

    cv2.imshow(window_name, frame)

    key = cv2.waitKey(1) & 0xFF
    
    if key == ord(' '):
      try:
        prediction = predict_picture(frame)
        label = labels[prediction]
        print(f"Predicted Label: #{prediction} ({label})")
        send_serial(serial_port, prediction);
      except Exception as e:
        print(e)
    if key == ord('q'):
      break
    
    # absolutely bongus window close detector
    try:
      cv2.getWindowProperty(window_name, 0)
    except:
      break

  cap.release()
  cv2.destroyAllWindows()



In [15]:
camera_id = 0
serial_port = "COM3"
serial_port = serial.Serial(serial_port, 9600)
print("Serial initialized")

print("Waiting for serial (2 seconds) ...")
time.sleep(2)
print("Serial should be ready")

capture_camera(camera_id, serial_port)
print("Quit")

serial_port.close()
print("Serial closed")

Serial initialized
Waiting for serial (2 seconds) ...
Serial should be ready
Press the space bar to capture an image. Press 'q' to exit.
Predicted Label: 0 (0 Yellow Circle)
Serial sent b'0'
Predicted Label: 0 (0 Yellow Circle)
Serial sent b'0'
Predicted Label: 2 (2 Yellow Square)
Serial sent b'2'
Predicted Label: 0 (0 Yellow Circle)
Serial sent b'0'
Predicted Label: 0 (0 Yellow Circle)
Serial sent b'0'
Predicted Label: 2 (2 Yellow Square)
Serial sent b'2'
Predicted Label: 2 (2 Yellow Square)
Serial sent b'2'
Predicted Label: 2 (2 Yellow Square)
Serial sent b'2'
Predicted Label: 2 (2 Yellow Square)
Serial sent b'2'
Predicted Label: 0 (0 Yellow Circle)
Serial sent b'0'
Quit
Serial closed
